## Advent of Code - Day 21

### Star 1

In [1]:
import re

In [2]:
class Cube:
    
    def __init__(self, corner_list, on):
        self.x = tuple(corner_list[0:2])
        self.y = tuple(corner_list[2:4])
        self.z = tuple(corner_list[4:])
        self.on = on
        
    def intersect(self, other):
        new_corner_list = []
        new_corner_list.extend(self._line_overlap(self.x, other.x))
        new_corner_list.extend(self._line_overlap(self.y, other.y))
        new_corner_list.extend(self._line_overlap(self.z, other.z))
        # Logic to determine if generated intersecting cuboid
        # should be "on" or "off". Tricky conditions like an "on" / "on"
        # intersection generated off cuboid to avoid double counting
        # volumes. But also obvious conditions, like an "on" on top of a
        # "off" generates an "on" intersecting cuboid.
        if False in new_corner_list:
            return False
        if self.on and not other.on:
            new_on = True
        if not self.on and other.on:
            new_on = False
        if self.on and other.on:
            new_on = False
        if not self.on and not other.on:
            new_on = True
        return Cube(new_corner_list, new_on)
    
    def _line_overlap(self, pair1, pair2):
        check1 = self._line_overlap1(pair1, pair2)
        check2 = self._line_overlap1(pair2, pair1)
        if not (check1 or check2):
            return [False]
        return check1 if check1 else check2
    
    def _line_overlap1(self, pair1, pair2):
        if pair1[0] >= pair2[0] and pair1[0] <= pair2[1]:
            return (pair1[0], min((pair1[1], pair2[1])))
        if pair1[1] >= pair2[0] and pair1[1] <= pair2[1]:
            return (max((pair1[0], pair2[0])), pair1[1])
        return False
        
    def volume(self):
        vol = self.x[1] - self.x[0] + 1
        vol *= self.y[1] - self.y[0] + 1
        vol *= self.z[1] - self.z[0] + 1
        
        return vol if self.on else -vol
        
    def __str__(self):
        return f"x: {self.x}, y: {self.y}, z: {self.z}, on={self.on}"

In [3]:
p1 = re.compile(r"[-\d]+")
p2 = re.compile(r"[ofn]+")
def load_data(fn, init_only=True):
    cubes = []
    with open(fn) as f:
        for line in f:
            corners = [int(x) for x in p1.findall(line)]
            on = True if p2.findall(line)[0] == 'on' else False
            if (max(corners) > 50 or min(corners) < -50) and init_only:
                continue
            cubes.append(Cube(corners, on))
        
    return cubes

### Star 1 & 2

In [4]:
# True for Star 1, False for Star 2
cubes = load_data("input_22_a.txt", init_only=False)

stack = []
for new_cube in cubes:
    add_to_stack = []
    for cube in stack:
        intersect_cube = new_cube.intersect(cube)
        if intersect_cube == False:
            continue
        add_to_stack.append(intersect_cube)
    stack.extend(add_to_stack)
    # Only add "on" cuboids to the stack, "off" cuboids make
    # their impact by the generated intersections in for-loop.
    if new_cube.on: 
        stack.append(new_cube)
        
vol = 0
for cube in stack:
    vol += cube.volume()

print(f"Volume of on cubes = {vol}")

Volume of on cubes = 1323862415207825
